In [1]:
# Name :     Jugal Pareshbhai Patel
# UTA ID :   1001769143


import os 
import string
import random
import re
import numpy 
from random import seed, shuffle
from math import log

#paths = ['Downloads/aclImdb/train/neg/',  'Downloads/aclImdb/train/pos/', 'Downloads/aclImdb/test/neg/', 'Downloads/aclImdb/test/pos/']
path_to_train_data = ['Downloads/aclImdb/train/neg/', 'Downloads/aclImdb/train/pos/' ]
path_to_test_data = ['Downloads/aclImdb/test/neg/' ,'Downloads/aclImdb/test/pos/']


In [2]:
def load_data(path):
    files = [path + file for file in os.listdir(path)]
    
    temp_data = []
    
    for file in files:
        with open(file, 'r', encoding = 'utf-8') as f:
            #for line in f.readlines():
            #    temp_data.append(line.replace('<br />', '').translate(str.maketrans('', '', string.punctuation)).lower())
            temp_data.append(f.read())
    return temp_data

def format_data(data):
    for i, d in enumerate(data):
        d = re.sub('[^0-9a-zA-Z]+', ' ', d.lower())
        data[i] = ' '.join(d.split())
    

In [3]:
train_data_list = [[], []]

for path in path_to_train_data:
    train_data_list[path_to_train_data.index(path)] = load_data(path)

train_Negative_Data, train_Positive_Data = train_data_list[0], train_data_list[1]

format_data(train_Negative_Data)
format_data(train_Positive_Data)


In [4]:
test_data_list = [[], []]

for path in path_to_test_data:
    test_data_list[path_to_test_data.index(path)] = load_data(path)

test_Negative_Data, test_Positive_Data = test_data_list[0], test_data_list[1]

format_data(test_Negative_Data)
format_data(test_Positive_Data)


In [5]:
print(len(train_Negative_Data))
print(len(train_Positive_Data))

12500
12500


In [6]:
train_Neg_Datalabel, train_Pos_Datalabel = [], []
test_Neg_Datalabel, test_Pos_Datalabel = [], []

train_Labeled_DataList = [train_Neg_Datalabel, train_Pos_Datalabel]
test_Labeled_DataList = [test_Neg_Datalabel, test_Pos_Datalabel]


In [7]:
def Label_working(data, outputList):
    for d in data:
        for x in d:
            outputList[data.index(d)].append([x, int(data.index(d))])
    

In [8]:
Label_working(train_data_list, train_Labeled_DataList)
Label_working(test_data_list, test_Labeled_DataList)

In [9]:
print(len(train_Neg_Datalabel))
print(len(train_Pos_Datalabel))

12500
12500


In [10]:
train_Data_labeled = train_Neg_Datalabel + train_Pos_Datalabel
test_Data_labeled = test_Neg_Datalabel + test_Pos_Datalabel

In [11]:
#.  a. Divide the dataset as train, development and test.

k = 0.75
train_Data, dev_Data = [], []
random.shuffle(train_Data_labeled)
train_Data = train_Data_labeled[:int(len(train_Data_labeled)*k)]
dev_Data = train_Data_labeled[int(len(train_Data_labeled)*k):]

test_Data = test_Data_labeled
random.shuffle(test_Data)

In [12]:

print('Size of Train dataset : ', len(train_Data))
print('Size of Development dataset : ', len(dev_Data))
print('Size of Test dataset : ', len(test_Data))

Size of Train dataset :  18750
Size of Development dataset :  6250
Size of Test dataset :  25000


In [13]:
#b.Build a vocabulary as list. 

def Vocab_buider(data):
    vocab = {} #words have 2 counts in order neg, pos
    
    for d in data:
        word_set = set()
        word_set = word_set.union(set([q for q in re.split(r'[\s|,|;|.|/|\[|\]|;|\!|?|\'|\\|\)|\(|\"|@|&|#|-|*|%|>|<|^|-]\s*',str(d[0]).replace('<br />', '').strip()) if q]))
        for word in word_set:
            if word not in vocab:
                vocab[word] = [0, 0]
            vocab[word][d[1]] += 1
                
    return vocab

def verify_occur_threshold(vocab, Occurance_count):
    for key in list(vocab.keys()):
        if vocab[key][0] + vocab[key][1] < Occurance_count:
            del vocab[key]
            
    return vocab

In [14]:
trainVocab_Directory_Full = Vocab_buider(train_Data)
trainVocab_Directory = verify_occur_threshold(trainVocab_Directory_Full ,5)
trainVocab_Directory

{'children': [372, 453],
 'and': [8993, 9131],
 'it': [8401, 8352],
 'great': [1525, 3170],
 'worry': [59, 52],
 'enjoyable': [186, 388],
 'either': [768, 479],
 'christopher': [119, 136],
 'performance': [613, 1107],
 'family': [559, 1030],
 'whom': [187, 256],
 'show': [1183, 1328],
 'you': [5294, 4969],
 'want': [1366, 977],
 'good': [3659, 3565],
 'difficult': [227, 269],
 'make': [2664, 1949],
 'like': [4694, 4044],
 'pleasing': [19, 32],
 'if': [4385, 3668],
 'cast': [1078, 1358],
 'kid': [359, 326],
 'lighthearted': [6, 21],
 'bored': [276, 108],
 'not': [5919, 5307],
 'parent': [42, 44],
 'bill': [145, 189],
 'thought': [1147, 1035],
 'kids': [498, 469],
 'that': [7781, 7455],
 'fits': [60, 101],
 'without': [1017, 1095],
 'movie': [6196, 5285],
 'entertain': [54, 66],
 'a': [9052, 9073],
 'younger': [132, 209],
 'tears': [69, 159],
 'story': [2568, 3171],
 'watch': [2096, 1993],
 'would': [3356, 2773],
 'this': [8658, 8335],
 'from': [4291, 4481],
 'was': [6387, 5733],
 'nothi

In [15]:
#c.Probability of the occurrence


def find_total_label_list_ofdata(data):
    result_list = [0, 0, len(data)]
    
    for single_data in data:
        result_list[single_data[1]] += 1
    
    return result_list
    

def Probability_of_Occurance(word, vocab_Directory, total_Count):
    if word not in vocab_Directory:
        return 0
    else:
        return ((vocab_Directory[word][0] + vocab_Directory[word][1])/total_Count)

# Conditional probability based on the sentiment
def conditional_Probability_of_word(word, vocab_Directory, label):
    temp_list = find_total_label_list_ofdata(train_Data)
    
    if word not in vocab_Directory:
        return 0
    elif int(label) == 0:
        return (vocab_Directory[word][0]/temp_list[0])
    elif int(label) == 1:
        return (vocab_Directory[word][1]/temp_list[1])

In [16]:
print('P["the"] = ', Probability_of_Occurance('the', trainVocab_Directory, len(train_Data)))
print('P["the"|Negative] = ', conditional_Probability_of_word('the', trainVocab_Directory, 0))
print('P["the"|Positive] = ', conditional_Probability_of_word('the', trainVocab_Directory, 1))

P["the"] =  0.9912533333333333
P["the"|Negative] =  0.9925229651783807
P["the"|Positive] =  0.989987217724755


In [17]:
#d. Calculate accuracy using dev dataset 


def make_prediction(rev, vocab, n_p_list, lambda_threshold, length_Of_Data):
    List_probability = list()
    
    for l in [0, 1]:
        single_prob = log(n_p_list[l]/length_Of_Data)
        
        for w in rev[0]:
            if w not in vocab:
                continue
            if lambda_threshold == 0 and vocab[w][l] == 0:
                single_prob = 0
                break
                
            single_prob += log((vocab[w][l] + lambda_threshold) / (vocab[w][0] + vocab[w][1] + lambda_threshold))
        
        List_probability.append(single_prob)
        
    return 0 if List_probability[0] > List_probability[1] else 1

def part_data_to_kfolds(empty_list, data, k):
    fold_Length = len(data)//k
    
    for i in range(k):
        shuffle(data)
        single_Fold = data[:fold_Length]
        data = data[fold_Length:]
        empty_list.append(single_Fold)
        

def cal_data_accu_for_kfolds(data, lambda_threshold, k, Occurance_count):
    List_accuracy = list()
    List_fold = list()
    
    Equivalent_data = list(data)
    
    part_data_to_kfolds(List_fold, Equivalent_data, k)
    
    #print(List_fold)
    
    for fold in List_fold:
        train_List_fold = list(List_fold)
        list(List_fold).remove(fold)
        train_List_fold = sum(train_List_fold, [])
        test_List_fold = fold
        
        vocab = verify_occur_threshold(Vocab_buider(train_List_fold), Occurance_count)
        
        total_label_list = find_total_label_list_ofdata(train_List_fold)
        
        counter = 0
        
        for rev in test_List_fold:
            if make_prediction(rev, vocab, total_label_list, lambda_threshold, len(train_List_fold)) == rev[1]:
                counter += 1
        
        List_accuracy.append(counter / len(test_List_fold))
        
    return List_accuracy
        

In [18]:
seed(1)

print('Accuracy for 5-fold: ', cal_data_accu_for_kfolds(dev_Data, 0, 5, 5))

Accuracy for 5-fold:  [0.5, 0.5224, 0.5248, 0.4792, 0.5104]


In [19]:
# e. calculate accuracy and find top 10 negative and positive words from dataset.

def cal_accuracy_function(train_dataset, test_dataset, lambda_threshold, Occurance_count):
    vocab = Vocab_buider(train_dataset)
    vocab = verify_occur_threshold(vocab, Occurance_count)
    
    total_label_list = find_total_label_list_ofdata(train_dataset)
    
    counter = 0
    
    for r in test_dataset:
        if make_prediction(r, vocab, total_label_list, lambda_threshold, len(train_dataset)) == r[1]:
            counter += 1
            
    return (counter / len(test_dataset))
        

In [20]:
print('Accuracy without smoothing: ', cal_accuracy_function(train_Data, dev_Data, 0, 5))

Accuracy without smoothing:  0.49152


In [21]:
print('Accuracy with laplace estimate: ', cal_accuracy_function(train_Data, dev_Data, 1, 5))

Accuracy with laplace estimate:  0.4912


In [22]:
def get_top_10_words(data, label, lambda_threshold, Occurance_count):
    vocab = Vocab_buider(data)
    vocab = verify_occur_threshold(vocab, Occurance_count)
    
    words = list()
    
    for word in list(vocab.keys()):
        temp = (vocab[word][label] + lambda_threshold) / (vocab[word][0] + lambda_threshold + vocab[word][1] + lambda_threshold)
        words.append([word, temp])
        
    words = numpy.array(words)
    words = words[numpy.lexsort(words.T)]
    
    return words
    

In [23]:
negative_words = get_top_10_words(dev_Data, 0, 1, 5)[:10]
positive_words = get_top_10_words(dev_Data, 1, 1, 5)[:10]

In [24]:
print(negative_words)

[['simplicity' '0.043478260869565216']
 ['walsh' '0.05555555555555555']
 ['technicolor' '0.0625']
 ['nuanced' '0.06666666666666667']
 ['1936' '0.07142857142857142']
 ['absorbing' '0.07142857142857142']
 ['excellently' '0.07142857142857142']
 ['raoul' '0.07142857142857142']
 ['conductor' '0.07692307692307693']
 ['heartwarming' '0.07692307692307693']]


In [25]:
print(positive_words)

[['drivel' '0.03225806451612903']
 ['incoherent' '0.034482758620689655']
 ['dreck' '0.03571428571428571']
 ['stinker' '0.05']
 ['nope' '0.058823529411764705']
 ['semblance' '0.058823529411764705']
 ['boll' '0.0625']
 ['incompetent' '0.0625']
 ['mercifully' '0.0625']
 ['redeeming' '0.0641025641025641']]


In [26]:
#f. Using the test dataset

end_accuracy = cal_accuracy_function(train_Data, test_Data, 1, 5)

In [27]:
end_accuracy

0.4968